In [5]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime
from datetime import date
import glob

from scipy.optimize import root_scalar
import scipy.optimize as opt

In [6]:
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)

In [13]:
def find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file

def clean_position(position):
    position_copy = position.copy()
    position_copy = position_copy[
        position_copy["Current Value"].notna()
    ]  # remove rows without current value
    position_copy["Current Value"] = transfer_dollar_to_float(
        position_copy["Current Value"]
    )
    position_copy["Cost Basis Total"] = transfer_dollar_to_float(
        position_copy["Cost Basis Total"]
    )
    return position_copy



def transfer_dollar_to_float(dat):
    """
    Change "$123,456" to 123456.0, and "--" to 0.0
    """
    # Replace any "--" with "$0"
    cleaned = dat.str.replace("--", "$0", regex=False)
    # Remove dollar sign and commas, then convert to float
    cleaned = cleaned.str.replace("$", "", regex=False).str.replace(",", "", regex=False)
    return cleaned.astype(float)


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)

    transactions = combine_transaction_files(transaction_files)
    transactions = clean_transactions(transactions)
    return transactions

def combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(14)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions


def clean_transactions(transactions):
    transactions_copy = transactions.copy()
    transactions_copy = transactions_copy[
        transactions_copy["Amount ($)"].notna()
    ]  # remove rows without  value
    transactions_copy["Run Date"] = pd.to_datetime(
        transactions_copy["Run Date"], format=" %m/%d/%Y"
    ).dt.date
    transactions_copy["Settlement Date"] = pd.to_datetime(
        transactions_copy["Settlement Date"], format="%m/%d/%Y"
    ).dt.date
    transactions_copy.loc[transactions_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    transactions_copy["Symbol"] = transactions_copy[
        "Symbol"
    ].str.lstrip()  # remove space at the beginning of Symbol
    transactions_copy = transactions_copy.sort_values(by="Run Date").reset_index(
        drop=True
    )
    return transactions_copy

In [14]:
## load position
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'


In [15]:
position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
position_files = glob.glob(position_file_path_pattern)
position_file = find_latest_position_file(position_files)
position = pd.read_csv(position_file)
position = clean_position(position)

In [16]:
transaction_file_path_pattern = os.path.join(
    data_folder_path, transaction_file_pattern
)
transaction_files = glob.glob(transaction_file_path_pattern)

transactions = combine_transaction_files(transaction_files)
transactions = clean_transactions(transactions)

In [17]:
class Portfolio:
    """
    A class to represent a fidelity portfolio.

    Attributes
    ----------
    transactions: pd.DataFrame
        All historial transactions.
    position: pd.DataFrame
        Current balance in the account.
    today: datetime.date
        Today's date
    individual_transactions: pd.DataFrame
        Transactions under account "Individual Z23390746"
    individual_position: pd.DataFrame
        Position under account "Individual Z23390746"

    Methods
    -------
    show_investment_distribution:
        display the result of get_investment_distribution.
    get_investment_distribution:
        get the distribution of investment amony differnet asset.
    get_total_investment:
        get total investment amount in dollar.
    get_stock_investment:
        get amount of stock investment in dollar.
    get_bill_investment:
        get amount of bill investment in dollar.
    get_other_investment:
        get amount of other investment in dollar.
    show_stock_irr:
        display the result of get_stock_irr.
    get_stock_irr:
        get the irr of each stock.
    add_total_current_value_to_individual_position:
        ---
    get_total_current_value:
        ---
    set_merged_individual_position_transaction:
        merge individual_position and individual_transaction.
    add_time_diff:
        ---
    calculate_irr(symbol_list, lower_bound=-0.999, upper_bound=5):
        filter all transactions with symbol in symbol_list. Then calculate the irr of these transactions.
    filter_merged_transactions_by_symbol(symbol_list):
        filter all transactions with symbol in symbol_list.
    """

    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position

        account_number_dic = {
            "Individual":'Z23390746',
            "401k":'86964',
            "HSA":'241802439',
            "Cash":'Z06872898'
        }
        
        self.today = datetime.now().date()
        self.individual_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["Individual"]
        ]
        self.pension_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["401k"]
        ]
        self.HSA_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["HSA"]
        ]
        self.cash_transactions = self.transactions[
            self.transactions["Account Number"] == account_number_dic["Cash"]
        ]
        
        self.individual_position = self.position[
            self.position["Account Number"] == account_number_dic["Individual"]
        ]
        self.pension_position = self.position[
            self.position["Account Number"] == account_number_dic["401k"]
        ]
        self.HSA_position = self.position[
            self.position["Account Number"] == account_number_dic["HSA"]
        ]
        self.cash_position = self.position[
            self.position["Account Number"] == account_number_dic["Cash"]
        ]
        

    def show_investment_distribution(self):
        investment_distribution = self.get_investment_distribution()
        investment_distribution["Percent"] = [
            f"{x * 100:.2f}%" for x in investment_distribution["Percent"]
        ]
        print(pd.DataFrame(investment_distribution))

    def get_investment_distribution(self):
        total_investment = self.get_total_investment()
        stock_investment = self.get_stock_investment()
        bill_investment = self.get_bill_investment()
        other_investment = self.get_other_investment()
        result = {
            "Class": ["stock", "bill", "other", "total"],
            "Amount": [
                stock_investment,
                bill_investment,
                other_investment,
                total_investment,
            ],
            "Percent": [
                stock_investment / total_investment,
                bill_investment / total_investment,
                other_investment / total_investment,
                total_investment / total_investment,
            ],
        }

        return result

    def get_total_investment(self):
        total_investment = self.individual_transactions[
            self.individual_transactions["Symbol"] == "Transfer"
        ]["Amount ($)"].sum()
        return total_investment

    def get_stock_investment(self):
        stock_position = self.individual_position[
            ~(self.individual_position["Description"] == "HELD IN MONEY MARKET")
            & ~(self.individual_position["Description"].str.contains("BILLS", na=False))
        ]
        return stock_position["Cost Basis Total"].sum()

    def get_bill_investment(self):
        bill_position = self.individual_position[
            (self.individual_position["Description"].str.contains("BILLS", na=False))
        ]
        return bill_position["Cost Basis Total"].sum()

    def get_other_investment(self):
        return (
            self.get_total_investment()
            - self.get_stock_investment()
            - self.get_bill_investment()
        )

    def show_stock_irr(self):
        stock_irr = self.get_stock_irr()
        stock_irr_nice_look = pd.DataFrame(
            {
                "Stock": stock_irr.keys(),
                "irr": [f"{(stock_irr[x]) * 100:.2f}%" for x in stock_irr.keys()],
            }
        )
        print(pd.DataFrame(stock_irr_nice_look))

    def get_stock_irr(self):
        self.add_total_current_value_to_individual_position()
        self.set_merged_individual_position_transaction()
        self.merged_individual_position_transaction = self.add_time_diff(self.merged_individual_position_transaction)

        unique_symbols = self.merged_individual_position_transaction["Symbol"].unique()
        stock_list = [
            element
            for element in unique_symbols
            if "FZFXX" not in element
            and not element[0].isdigit()
            and "Pending" not in element
        ]
        result_dict = {x: self.calculate_irr(self.filter_merged_transactions_by_symbol([x])) for x in stock_list}
        if "Transfer" in stock_list:
            stock_list.remove("Transfer")
        transactions = self.filter_merged_transactions_by_symbol(stock_list)
        result_dict["stock"] = self.calculate_irr(transactions)

        return result_dict

    def add_total_current_value_to_individual_position(self):
        if "Transfer" in self.individual_position["Symbol"].values:
            print("Total current value has been added")
            return

        total_current_value = self.get_total_current_value()
        new_rows = pd.DataFrame(
            {"Symbol": ["Transfer"], "Current Value": -total_current_value}
        )
        self.individual_position = pd.concat(
            [self.individual_position, new_rows], ignore_index=True
        )

    def get_total_current_value(self):
        return self.individual_position["Current Value"].sum()

    def set_merged_individual_position_transaction(self):
        new_rows = pd.DataFrame(
            {
                "Run Date": [self.today] * len(self.individual_position),
                "Symbol": self.individual_position["Symbol"],
                "Amount ($)": self.individual_position["Current Value"],
            }
        )

        self.merged_individual_position_transaction = pd.concat(
            [self.individual_transactions, new_rows], ignore_index=True
        )

    def add_time_diff(self, transactions):
        transactions_copy = transactions.copy()
        if not 'time_diffs_in_year' in transactions_copy.columns:
            transactions_copy.loc[:, "time_diffs_in_year"] = (
                self.today - transactions_copy.loc[:, "Run Date"]
            ).apply(lambda x: x.days) / 365.25
        else:
            print("Column time_diffs_in_year exists")
        return transactions_copy
        

    def calculate_irr(self, transactions, lower_bound=-0.999, upper_bound=5):

        def npv(rate):
            return np.sum(
                transactions["Amount ($)"]
                * (1 + rate) ** transactions["time_diffs_in_year"]
            )

        result = root_scalar(npv, bracket=[lower_bound, upper_bound], method="bisect")

        if result.converged:
            return result.root
        else:
            raise ValueError(
                "No solution found for the rate that satisfies the equation."
            )

    def filter_merged_transactions_by_symbol(self, symbol_list):
        return self.merged_individual_position_transaction[
            self.merged_individual_position_transaction["Symbol"].isin(symbol_list)
        ]
        
    

In [18]:
current_portfolio = Portfolio(transactions=transactions, position=position)

In [19]:
current_portfolio.transactions[
            current_portfolio.transactions["Account"] == "Individual"
        ]

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
0,2022-07-26,Individual,Z23390746,Electronic Funds Transfer Received (Cash),Transfer,No Description,Cash,0.00,NaN,NaN,NaN,NaN,1000.00,NaT
1,2022-07-29,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,0.22,NaT
2,2022-07-29,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.22,1.00,NaN,NaN,NaN,-0.22,NaT
3,2022-08-05,Individual,Z23390746,YOU BOUGHT STARBUCKS CORP COM USD0.001 (SBUX)...,SBUX,STARBUCKS CORP COM USD0.001,Cash,1.00,85.20,NaN,NaN,NaN,-85.20,2022-08-09
4,2022-08-05,Individual,Z23390746,YOU BOUGHT STARBUCKS CORP COM USD0.001 (SBUX)...,SBUX,STARBUCKS CORP COM USD0.001,Cash,9.00,85.36,NaN,NaN,NaN,-768.20,2022-08-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,2025-04-30,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,336.77,1.00,NaN,NaN,NaN,-336.77,NaT
508,2025-04-30,Individual,Z23390746,DIVIDEND RECEIVED JPMORGAN CHASE &CO. COM (JP...,JPM,JPMORGAN CHASE &CO. COM,Cash,0.00,NaN,NaN,NaN,NaN,14.00,NaT
509,2025-05-01,Individual,Z23390746,REDEMPTION PAYOUT UNITED STATES TREAS BILLS Z...,912797ND5,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 0...,Cash,-150000.00,1.00,NaN,NaN,NaN,150000.00,NaT
510,2025-05-01,Individual,Z23390746,DIVIDEND RECEIVED VERIZON COMMUNICATIONS INC ...,VZ,VERIZON COMMUNICATIONS INC,Cash,0.00,NaN,NaN,NaN,NaN,33.88,NaT


In [20]:
current_portfolio.individual_transactions[
            current_portfolio.individual_transactions["Symbol"] == "Transfer"
        ]["Amount ($)"]

0        1000.0
7       29000.0
91      30000.0
100     40000.0
142      1500.0
161    148500.0
163   -148500.0
164    148500.0
172    100000.0
177   -100000.0
180    100000.0
181   -100000.0
184    140000.0
190     50000.0
195     10000.0
211     50000.0
268     50000.0
302     50000.0
368    150000.0
412     50000.0
415     50000.0
419     50000.0
439     50000.0
452     25500.0
453     24500.0
458     50000.0
Name: Amount ($), dtype: float64

In [21]:
current_portfolio.get_total_investment()

np.float64(1050000.0)

In [22]:
[current_portfolio.get_stock_investment(),current_portfolio.get_bill_investment(),current_portfolio.get_other_investment()]

[np.float64(362051.45999999996),
 np.float64(546222.19),
 np.float64(141726.3500000001)]

In [23]:
current_portfolio.get_investment_distribution()

{'Class': ['stock', 'bill', 'other', 'total'],
 'Amount': [np.float64(362051.45999999996),
  np.float64(546222.19),
  np.float64(141726.3500000001),
  np.float64(1050000.0)],
 'Percent': [np.float64(0.34481091428571425),
  np.float64(0.5202116095238095),
  np.float64(0.1349774761904763),
  np.float64(1.0)]}

In [24]:
current_portfolio.show_investment_distribution()

   Class      Amount  Percent
0  stock   362051.46   34.48%
1   bill   546222.19   52.02%
2  other   141726.35   13.50%
3  total  1050000.00  100.00%


In [25]:
current_portfolio.show_stock_irr()

       Stock      irr
0   Transfer    6.20%
1       SBUX    3.16%
2       AAPL   11.02%
3        JPM   34.64%
4        AXP   27.10%
5      GOOGL   30.74%
6       AMZN   35.82%
7        NKE  -23.91%
8       TSLA   34.16%
9      FXAIX    7.88%
10       MCD   26.03%
11     FSPSX   12.65%
12     FSKAX    1.20%
13      COKE   10.63%
14      MSFT   -1.61%
15      BRKB   22.28%
16       KHC  -17.09%
17        VZ   22.74%
18      NVDA  -13.95%
19       PEP  -30.91%
20     stock    8.89%
